In [3]:
import config
import requests
import datetime as dt
import json
from datetime import datetime as dt, date
from dateutil.relativedelta import relativedelta
import pandas as pd
import requests
import os
import sys

API_KEY = config.API_KEY


# Symbol lists contain all the stocks that we are interested in. 
symbol_list = ['AAPL','MSFT','GOOG','AMZN','NVDA','BRK-B','TSLA','META','JNJ','V']

In [2]:

def get_daily_prices(symbol):
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&outputsize=full&apikey={API_KEY}'
    r = requests.get(url)
    
    data = r.json()

    if r.status_code == 200:
       
        price_data = data["Time Series (Daily)"]

        dates = []
        full_dates = list(price_data.keys())
    
        today = date(2020,2,1)
   
    
        prices = []
        for i in range(0, len(full_dates)):
            day = dt.strptime(full_dates[i], "%Y-%m-%d").date()
            if day < today - relativedelta(years=10):
                break
            else:
                dates.append(full_dates[i])
                prices.append(float(price_data[full_dates[i]]["4. close"]))

        his_prices = {"dates": dates, "prices": prices}
    
        return his_prices
    else: return None

In [3]:
#import pandas_datareader as pdr

#symbols = pdr.get_nasdaq_symbols()
#symbols.head()


In [4]:
his_prices = get_daily_prices('META')


In [5]:
# A template method for showing one data for all stocks

df_incremental = []
for symbol in symbol_list:
    temp = get_daily_prices(symbol)
    temp2 = dict()
    temp2["date"] = temp["dates"]
    temp2[symbol] = temp["prices"]
    temp_df = pd.DataFrame.from_dict(temp2)
    df_incremental.append(temp_df)

list_of_dates = [df_incremental[i]["date"].tolist() for i in range(len(df_incremental))]
unique_dates = sorted(list(set([item for sublist in list_of_dates for item in sublist])))
df= pd.DataFrame(unique_dates, columns=["date"])

for i in range(0, len(df_incremental)):
    df = pd.merge(df, df_incremental[i], on="date")

#df # also works if there are missing dates for some stocks even first one
df['date'] = pd.to_datetime(df['date'])

# Set the date column as the index
df.set_index('date', inplace=True)

# Resample the data to weekly frequency and calculate the mean price
df_weekly = df.resample('W').mean()  # W for weekly, M for monthly, A for annual

# Print the weekly data
print(df_weekly)


                  AAPL      MSFT     GOOG     AMZN     NVDA    BRK-B   
date                                                                   
2014-03-30  537.160000   39.8300  559.225  338.380   17.845  123.500  \
2014-04-06  538.310000   40.9280  560.802  335.587   18.394  124.506   
2014-04-13  524.064000   39.7318  545.748  321.095   18.482  122.846   
2014-04-20  520.897475   39.8350  540.400  320.145   18.455  124.810   
2014-04-27  545.465800   39.8780  526.342  325.150   18.932  127.012   
...                ...       ...      ...      ...      ...      ...   
2023-03-19  153.380000  267.1560   97.198   96.500  245.046  300.322   
2023-03-26  158.738000  275.3060  104.862   98.772  265.074  300.386   
2023-04-02  160.792000  280.8940  102.328  100.164  270.170  304.722   
2023-04-09  165.055000  287.5875  105.970  102.380  273.340  310.570   
2023-04-16  162.740000  286.3380  107.188  100.966  268.928  315.732   

                TSLA       META       JNJ         V  
date     

In [6]:
# 1. SMA 

def calculate_sma(df, symbols, window):
    '''
    This function calculates the simple moving average (SMA) of closing prices
    for multiple stock symbols over a given time window.
    
    Parameters:
    df (Pandas DataFrame): The input DataFrame containing the stock data
    symbols (list): A list of stock symbols for which to calculate the SMA
    window (int): The time window over which to calculate the SMA
    
    Returns:
    sma_df (Pandas DataFrame): A DataFrame containing the SMA of closing prices
    for all the specified stock symbols and time window.
    '''
    
    sma_df = df  # Initialize an empty DataFrame to store the results
    
    for symbol in symbols:
        sma_df[symbol+"_SMA"] = df[symbol].rolling(window=window).mean()
    
    return sma_df[ [x+"_SMA" for x in symbols] ]

calculate_sma(df, symbol_list, 20)

,AAPL_SMA,MSFT_SMA,GOOG_SMA,AMZN_SMA,NVDA_SMA,BRK-B_SMA,TSLA_SMA,META_SMA,JNJ_SMA,V_SMA
date,,,,,,,,,,
2014-03-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-03-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-04-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2023-04-10,159.3400,277.8260,102.6385,99.4435,263.5300,304.1025,188.8560,204.1025,155.3815,221.9310
2023-04-11,159.8565,279.2715,103.3615,99.8180,265.6315,304.6435,189.4715,205.7500,155.9420,222.6300
2023-04-12,160.2320,280.4065,103.9100,99.9655,266.8475,305.1045,189.3355,206.7490,156.4420,223.0875


In [7]:
# 2. EMA 

def calculate_ema(df, symbols, window):
    '''
    This function calculates the simple moving average (SMA) of closing prices
    for multiple stock symbols over a given time window.
    
    Parameters:
    df (Pandas DataFrame): The input DataFrame containing the stock data
    symbols (list): A list of stock symbols for which to calculate the SMA
    window (int): The time window over which to calculate the SMA
    
    Returns:
    sma_df (Pandas DataFrame): A DataFrame containing the SMA of closing prices
    for all the specified stock symbols and time window.
    '''
    
    sma_df = df  # Initialize an empty DataFrame to store the results
    
    for symbol in symbols:
        sma_df[symbol+"_EMA"] = df[symbol].ewm(span=window, adjust=False).mean()
    
    return sma_df[ [x+"_EMA" for x in symbols] ]

calculate_ema(df, symbol_list, 20)

,AAPL_EMA,MSFT_EMA,GOOG_EMA,AMZN_EMA,NVDA_EMA,BRK-B_EMA,TSLA_EMA,META_EMA,JNJ_EMA,V_EMA
date,,,,,,,,,,
2014-03-27,537.460000,39.360000,558.460000,338.470000,17.790000,123.470000,207.320000,60.970000,97.280000,215.770000
2014-03-28,537.402857,39.449524,558.605714,338.452857,17.800476,123.475714,207.800952,60.878571,97.295238,215.419524
2014-03-31,537.339728,39.596236,558.449932,338.254014,17.810907,123.618027,207.862766,60.817755,97.384263,215.461474
2014-04-01,537.750230,39.769928,559.279462,338.705060,17.890821,123.738215,208.730122,60.989397,97.437190,215.388953
2014-04-02,538.207351,39.920411,560.014752,339.015054,17.951695,123.807909,210.783444,61.154217,97.512696,215.319529
...,...,...,...,...,...,...,...,...,...,...
2023-04-10,159.801485,278.350980,102.734707,99.842399,263.032946,306.706677,189.903910,203.747661,157.266852,223.717232
2023-04-11,159.896581,278.777553,103.057116,99.849789,263.857427,307.372708,189.607347,204.709789,157.933818,224.167972
2023-04-12,159.915955,279.226358,103.263105,99.657428,263.961482,308.056260,188.743790,205.594571,158.503931,224.514832


In [8]:
def get_daily_adjusted_prices(symbol):
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&outputsize=full&apikey={API_KEY}'
    
    r = requests.get(url)
    #data_adjusted = r.json()
    data = r.json()
  
    if   r.status_code==200:
       
        #price_data_adj = data_adjusted["Time Series (Daily)"]
        price_data = data["Time Series (Daily)"]
        price_df = pd.DataFrame(price_data).transpose().reset_index()
        price_df.rename(columns={'index':'Date'},inplace=True)
        #price_df_adj = pd.DataFrame(price_data_adj).reset_index()
        #price_df_adj.rename(columns={'index':'Date'},inplace=True)
        #price_df = price_df.merge(price_df_adj.loc[:,["Date","5. adjusted close"]], on='Date',how='left')
        price_df.loc[:,'Date'] = [dt.strptime(d,'%Y-%m-%d') for d in price_df.Date]
        end_date = dt(2020,2,1)
        start_date = dt(2010,1,1)
        price_df = price_df.loc[(price_df.Date>=start_date)&(price_df.Date<=end_date),:].reset_index(drop=True)
        return price_df
    else: return None

def get_monthly_adjusted_prices(symbol):
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY_ADJUSTED&symbol={symbol}&outputsize=full&apikey={API_KEY}'
    
    r = requests.get(url)
    data = r.json()
  
    if   r.status_code==200:
        price_data = data["Monthly Adjusted Time Series"]
        price_df = pd.DataFrame(price_data).transpose().reset_index()
        price_df.rename(columns={'index':'Date'},inplace=True)

        price_df.loc[:,'Date'] = [dt.strptime(d,'%Y-%m-%d') for d in price_df.Date]
        end_date = dt(2020,2,1)
        start_date = dt(2010,1,1)
        price_df = price_df.loc[(price_df.Date>=start_date)&(price_df.Date<=end_date),:].reset_index(drop=True)
        return price_df
    else: return None

def get_weekly_adjusted_prices(symbol):
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY_ADJUSTED&symbol={symbol}&outputsize=full&apikey={API_KEY}'
    
    r = requests.get(url)
    data = r.json()
  
    if   r.status_code==200:
        price_data = data["Weekly Adjusted Time Series"]
        price_df = pd.DataFrame(price_data).transpose().reset_index()
        price_df.rename(columns={'index':'Date'},inplace=True)

        price_df.loc[:,'Date'] = [dt.strptime(d,'%Y-%m-%d') for d in price_df.Date]
        end_date = dt(2020,2,1)
        start_date = dt(2010,1,1)
        price_df = price_df.loc[(price_df.Date>=start_date)&(price_df.Date<=end_date),:].reset_index(drop=True)
        return price_df
    else: return None



,Date,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient
0,2020-01-31 00:00:00,208.43,208.69,201.06,201.91,201.91,31359917,0.0000,1.0
1,2020-01-30 00:00:00,206.53,211.43,204.75,209.53,209.53,48775500,0.0000,1.0
2,2020-01-29 00:00:00,221.44,224.2,218.45,223.23,223.23,33610005,0.0000,1.0
3,2020-01-28 00:00:00,216.14,218.74,213.52,217.79,217.79,13413835,0.0000,1.0
4,2020-01-27 00:00:00,213.1,216.06,212.5,214.87,214.87,13810655,0.0000,1.0
...,...,...,...,...,...,...,...,...,...
1933,2012-05-24 00:00:00,32.95,33.21,31.77,33.03,33.03,50237200,0.0000,1.0
1934,2012-05-23 00:00:00,31.37,32.5,31.36,32.0,32.0,73600000,0.0000,1.0
1935,2012-05-22 00:00:00,32.61,33.59,30.94,31.0,31.0,101786600,0.0000,1.0
1936,2012-05-21 00:00:00,36.53,36.66,33.0,34.03,34.03,168192700,0.0000,1.0


In [9]:

# 3. STOCH

def calculate_stoch(symbol, k_window=14, d_window=3): 
    
    # Filter the DataFrame to include only the rows with the specified symbol
    symbol_df = get_daily_adjusted_prices(symbol)
    
    # Set the date column as the DataFrame index
    symbol_df = symbol_df.set_index('Date')
    symbol_df['4. close'] = pd.to_numeric(symbol_df['4. close'])

    # Calculate the lowest and highest closing prices over the past k_window days
    low_min = symbol_df['3. low'].rolling(window=k_window).min()
    high_max = symbol_df['2. high'].rolling(window=k_window).max()
    
    # Calculate the %K line using the current closing price and the recent trading range
    k = 100 * (symbol_df['4. close']) - low_min / (high_max - low_min)
    
    # Calculate the %D line using a moving average of the %K line
    d = k.rolling(window=d_window).mean()
    
    # Combine the %K and %D lines into a DataFrame
    stoch_df = pd.DataFrame({'%K': k, '%D': d})
    
    return stoch_df

calculate_stoch('META')

,%K,%D
Date,,
2020-01-31,NaN,NaN
2020-01-30,NaN,NaN
2020-01-29,NaN,NaN
2020-01-28,NaN,NaN
2020-01-27,NaN,NaN
...,...,...
2012-05-24,3299.681404,3122.370179
2012-05-23,3196.681404,3227.976028
2012-05-22,3096.837670,3197.733493


In [10]:
# 4. RSI

def calculate_rsi(symbol, period=14):
    """
    Calculates the relative strength index (RSI) of a DataFrame.

    Parameters:
    period (int): Number of periods to use for the RSI calculation.

    Returns:
    pandas.DataFrame: DataFrame containing the RSI values.
    """
    df = get_daily_adjusted_prices(symbol)
    df["4. close"] = pd.to_numeric(df["4. close"])
    delta = df['4. close'].diff()

    # Get upward and downward price movements
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    # Calculate the exponential moving average (EMA) of gains and losses
    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()

    # Calculate the relative strength (RS)
    rs = avg_gain / avg_loss

    # Calculate the RSI
    rsi = 100 - (100 / (1 + rs))

    # Add RSI values to the DataFrame
    df[symbol + '_rsi'] = rsi

    return df.loc[:,["Date", symbol + '_rsi']]

calculate_rsi('META')

,Date,META_rsi
0,2020-01-31 00:00:00,NaN
1,2020-01-30 00:00:00,NaN
2,2020-01-29 00:00:00,NaN
3,2020-01-28 00:00:00,NaN
4,2020-01-27 00:00:00,NaN
...,...,...
1933,2012-05-24 00:00:00,67.109443
1934,2012-05-23 00:00:00,67.061030
1935,2012-05-22 00:00:00,62.218300
1936,2012-05-21 00:00:00,70.225140


In [11]:
# 5. MFI
import numpy as np

def calculate_mfi(symbol, period=14):
    """
    Calculates the money flow index (MFI) of a DataFrame.

    Parameters:
    period (int): Number of periods to use for the MFI calculation.

    Returns:
    pandas.DataFrame: DataFrame containing the MFI values.
    """
    df = get_daily_adjusted_prices(symbol)
    df["4. close"] = pd.to_numeric(df["4. close"])
    df["3. low"] = pd.to_numeric(df["3. low"])
    df["2. high"] = pd.to_numeric(df["2. high"])
    df["6. volume"] = pd.to_numeric(df["6. volume"])
    # Calculate typical price
    typical_price = (df['4. close'] + df['3. low'] + df['2. high']) / 3

    # Calculate raw money flow
    money_flow = typical_price * df['6. volume']

    # Get positive and negative money flow
    positive_flow = np.where(typical_price > typical_price.shift(1), money_flow, 0)
    negative_flow = np.where(typical_price < typical_price.shift(1), money_flow, 0)

    # Calculate the money flow ratio
    raw_mfr = positive_flow / negative_flow
    mfr = pd.Series(np.where(negative_flow == 0, 100, 100 - (100 / (1 + raw_mfr))))

    # Calculate the MFI
    mfi = mfr.rolling(window=period).mean()

    # Add MFI values to the DataFrame
    df[symbol + '_mfi'] = mfi

    return df.loc[:,["Date", symbol + '_mfi']]

calculate_mfi('META')

/tmp/ipykernel_243/423447993.py:30: RuntimeWarning: divide by zero encountered in divide
  raw_mfr = positive_flow / negative_flow
/tmp/ipykernel_243/423447993.py:30: RuntimeWarning: invalid value encountered in divide
  raw_mfr = positive_flow / negative_flow


,Date,META_mfi
0,2020-01-31 00:00:00,NaN
1,2020-01-30 00:00:00,NaN
2,2020-01-29 00:00:00,NaN
3,2020-01-28 00:00:00,NaN
4,2020-01-27 00:00:00,NaN
...,...,...
1933,2012-05-24 00:00:00,42.857143
1934,2012-05-23 00:00:00,42.857143
1935,2012-05-22 00:00:00,42.857143
1936,2012-05-21 00:00:00,50.000000


The parabolic SAR (Stop and Reverse) is a technical analysis indicator used to determine the direction of an asset's momentum and to provide potential entry and exit points for trades. It is calculated using a series of values and conditions based on the direction of the asset's price movement.

Here is a general formula for calculating the parabolic SAR:

- Initialize the parabolic SAR for the first day as the lowest low over a given lookback period. If there are no previous lows, use the lowest low for the current lookback period.
- For subsequent days, calculate the parabolic SAR as follows:
  - If the previous day's parabolic SAR was bullish (below the price), use the previous day's parabolic SAR value as the starting value for the current day's calculation.
  - If the previous day's parabolic SAR was bearish (above the price), use the highest high over the given lookback period as the starting value for the current day's calculation.
  - Calculate the acceleration factor (AF) based on the number of consecutive bullish or bearish days.
  - Calculate the new parabolic SAR using the previous day's parabolic SAR value, the acceleration factor, and the difference between the previous day's parabolic SAR value and the lowest low over the given lookback period.
  - If the new parabolic SAR value is higher than the current day's low, set the parabolic SAR equal to the current day's low.
  - If the new parabolic SAR value is lower than the current day's high, set the parabolic SAR equal to the current day's high.
  - If the price moves above or below the current day's parabolic SAR value, reverse the direction of the SAR and use the previous day's high or low as the starting value for the next calculation.



In [19]:
# 6. SAR 

def calculate_sar(symbol, lookback=14, acceleration_factor=0.02):
    # Load the price data into a pandas DataFrame
    df = get_daily_adjusted_prices(symbol)
    print(len(df))
    df["close"] = pd.to_numeric(df["4. close"])
    df["low"] = pd.to_numeric(df["3. low"])
    df["high"] = pd.to_numeric(df["2. high"])
    print(len(df))
    # Initialize the SAR for the first day as the lowest low over the lookback period
    sar = df['low'].rolling(window=lookback).min()
    trend = pd.Series(np.zeros(len(df)))

    # Loop through the data and calculate the SAR and trend for each day
    for i in range(lookback, len(df)):
        if df['low'][i] > sar[i-1]:
            trend[i] = 1
        elif df['high'][i] < sar[i-1]:
            trend[i] = -1
        else:
            trend[i] = trend[i-1]

        if trend[i] == 1:
            if sar[i-1] == df['low'][i-1]:
                sar[i] = sar[i-1] + acceleration_factor * (df['high'][i-1] - sar[i-1])
            else:
                sar[i] = sar[i-1] + acceleration_factor * (df['high'][i-1] - sar[i-1]) * trend[i-1]
            if sar[i] > df['low'][i]:
                sar[i] = df['low'][i]
        else:
            if sar[i-1] == df['high'][i-1]:
                sar[i] = sar[i-1] - acceleration_factor * (sar[i-1] - df['low'][i-1])
            else:
                sar[i] = sar[i-1] + acceleration_factor * (df['low'][i-1] - sar[i-1]) * trend[i-1]
            if sar[i] < df['high'][i]:
                sar[i] = df['high'][i]

    # Create a new DataFrame with the calculated Parabolic SAR values and the original price data
    sar_df = pd.DataFrame({'Parabolic SAR': sar}, index=df['Date'])
    
    return sar_df

calculate_sar('META', lookback=14, acceleration_factor=0.02)

1938
1938


,Parabolic SAR
Date,
2020-01-31,NaN
2020-01-30,NaN
2020-01-29,NaN
2020-01-28,NaN
2020-01-27,NaN
...,...
2012-05-24,NaN
2012-05-23,NaN
2012-05-22,NaN


In [20]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

calculate_sar('META').iloc[0:30,:]

1938
1938


,Parabolic SAR
Date,
2020-01-31,NaN
2020-01-30,NaN
2020-01-29,NaN
2020-01-28,NaN
2020-01-27,NaN
2020-01-24,NaN
2020-01-23,NaN
2020-01-22,NaN
2020-01-21,NaN


In [14]:
def calc_AD(df):
    # Initialize the AD line as the closing price
    df = get_daily_adjusted_prices(symbol)
    df["close"] = pd.to_numeric(df["4. close"])
    df["low"] = pd.to_numeric(df["3. low"])
    df["high"] = pd.to_numeric(df["2. high"])
    df["volume"] = pd.to_numeric(df["6. volume"])
    ad = df['close'].copy()
    
    # Calculate the money flow multiplier (MFM)
    mfm = ((df['close'] - df['low']) - (df['high'] - df['close'])) / (df['high'] - df['low'])
    
    # Calculate the money flow volume (MFV)
    mf_volume = mfm * df['volume']
    
    # Calculate the cumulative sum of the money flow volume
    cmf = mf_volume.cumsum()
    
    # Calculate the AD line
    ad.iloc[0] = cmf.iloc[0]
    for i in range(1, len(ad)):
        if df['close'].iloc[i] > df['close'].iloc[i-1]:
            ad.iloc[i] = ad.iloc[i-1] + cmf.iloc[i]
        elif df['close'].iloc[i] < df['close'].iloc[i-1]:
            ad.iloc[i] = ad.iloc[i-1] - cmf.iloc[i]
        else:
            ad.iloc[i] = ad.iloc[i-1]
            
    # Set the index of the DataFrame to the date column
    ad.index = df['Date']
    
    return pd.DataFrame(ad)

calc_AD('META')

,close
Date,
2020-01-31,-1.254954e+07
2020-01-30,-1.753785e+07
2020-01-29,-1.521235e+07
2020-01-28,-1.316688e+07
2020-01-27,-1.273729e+07
...,...
2010-01-08,-9.289570e+10
2010-01-07,-9.342936e+10
2010-01-06,-9.395936e+10


In [15]:
# -2: VMAP

def calculate_vwap(symbol, period):
    df = get_daily_adjusted_prices(symbol)
    df["close"] = pd.to_numeric(df["4. close"])
    df["low"] = pd.to_numeric(df["3. low"])
    df["high"] = pd.to_numeric(df["2. high"])
    df["volume"] = pd.to_numeric(df["6. volume"])

    # Calculate the typical price for each period
    tp = (df['high'] + df['low'] + df['close']) / 3

    # Multiply the typical price by the volume for each period
    vtp = tp * df['volume']

    # Calculate the cumulative sum of the volume times the typical price
    cum_vtp = vtp.cumsum()

    # Calculate the cumulative sum of the volume
    cum_vol = df['volume'].cumsum()

    # Calculate the VWAP for each period
    vwap = cum_vtp / cum_vol

    # Create a DataFrame to hold the results
    results = pd.DataFrame({
        'vwap': vwap[period-1:],
        'date': df['Date'][period-1:]
    })

    # Set the index to the start time
    results.set_index('date', inplace=True)

    return results

calculate_vwap('META', 14)


,vwap
date,
2020-01-13,215.947281
2020-01-10,216.061476
2020-01-09,216.133866
2020-01-08,216.067220
2020-01-07,215.922838
...,...
2012-05-24,87.002623
2012-05-23,86.938741
2012-05-22,86.850461


In [16]:
# MACD 

def calculate_macd(symbol, ema_short=12, ema_long=26, signal_period=9):
    df = get_daily_adjusted_prices(symbol)
    df["close"] = pd.to_numeric(df["4. close"])
    # Calculate the short-term and long-term EMAs
    ema_short = df['close'].ewm(span=ema_short, adjust=False).mean()
    ema_long = df['close'].ewm(span=ema_long, adjust=False).mean()
    
    # Calculate the MACD line and the signal line
    macd_line = ema_short - ema_long
    signal_line = macd_line.ewm(span=signal_period, adjust=False).mean()
    
    # Calculate the MACD histogram
    macd_histogram = macd_line - signal_line
    
    # Return a DataFrame with the MACD values
    macd_df = pd.DataFrame({
        'MACD': macd_line,
        'Signal': signal_line,
        'Histogram': macd_histogram
    }, index=df.index)
    
    return macd_df

calculate_macd('META')

,MACD,Signal,Histogram
0,0.000000,0.000000,0.000000
1,0.607863,0.121573,0.486291
2,2.170060,0.531270,1.638790
3,2.935313,1.012079,1.923234
4,3.268484,1.463360,1.805125
...,...,...,...
1933,0.334636,-0.110367,0.445004
1934,0.514460,0.014598,0.499862
1935,0.569713,0.125621,0.444092
1936,0.848219,0.270141,0.578078
